## Calculate Production Activity at CoolDown T = 0
---
A = Prodx_Rate * (1-exp(-0.693/halflife * beam_time)) * Stop_Ratio
+ Everything in the equation above is in seconds

In [1]:
import pdfplumber
import numpy as np
import os
import re
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Alignment

In [2]:
def Cal_Prodx_A(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames

    df = sheets["Initial"]
    beam_col_index = df.columns.get_loc('Beam')
    run_time_row_idx = df[df.iloc[:, beam_col_index + 2] == 'sec'].index[0]
    run_time = df.iloc[run_time_row_idx, beam_col_index + 1]

    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            if "Prodx A(Bq)" not in df.columns:
                df["Prodx_A(Bq)"] = -1.0
            df.at[df.index[-1], "Prodx A(Bq)"] = np.nan
            for idx in range(len(df)-1):
                prodx_rate = df.at[idx,'Prodx_Rate(pps)']
                halflife = df.at[idx,'t1/2(s)']
                stop_ratio = df.at[idx,'Stop_Ratio']
                df.at[idx,"Prodx_A(Bq)"] = prodx_rate * (1 - np.exp(-0.693/halflife * run_time)) * stop_ratio

            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

## Summarize sub-sheets

In [8]:
def summarize(excel_path, clean_table = True):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)

    # Extract data from sheets named "*um_*MeV"
    data_frames = []
    for sheet_name in workbook.sheetnames:
        if 'um_' in sheet_name and 'MeV' in sheet_name:
            df = sheets[sheet_name][['A', 'isotope', 't1/2', 'unit', 'Prodx_A(Bq)']]
            df.columns = ['A', 'isotope', 't1/2', 'unit', f'A(Bq)@{sheet_name.split("um_")[0]}um']
            data_frames.append((int(sheet_name.split('um_')[0]), df))

    # Sort data_frames by the numeric value in the sheet names
    data_frames.sort(key=lambda x: x[0])

    # Extract only the DataFrames in the correct order
    data_frames = [df for _, df in data_frames]

    # Merge all data frames on 'A' and 'isotope'
    merged_df = data_frames[0]
    for df in data_frames[1:]:
        merged_df = pd.merge(merged_df, df, on=['A', 't1/2', 'unit', 'isotope'], how='outer')

    # Reorder by "A" from large to small, and then by "isotope" keyword
    merged_df['isotope_keyword'] = merged_df['isotope'].str.replace(' isomer', '')
    merged_df = merged_df.sort_values(by=['A', 'isotope_keyword', 'isotope'], ascending=[False, True, True])
    # Drop the temporary column used for sorting
    merged_df = merged_df.drop(columns=['isotope_keyword'])
    
    # Round all "Prodx" columns to 2 decimal places
    for col in merged_df.columns:
        if "um" in col:
            merged_df[col] = merged_df[col].fillna(0).round(2)

    # Add the new column "A(Bq)@cd = 0s" if it doesn't exist
    if "A(Bq)@cd = 0s" not in merged_df.columns:
        merged_df["A(Bq)@cd = 0s"] = -1

    # Calculate the sum of all "A(Bq)@*um" columns for each row
    sum_columns = [col for col in merged_df.columns if "um" in col]
    merged_df["A(Bq)@cd = 0s"] = merged_df[sum_columns].sum(axis=1)

    # Remove rows where "A(Bq)@cd = 0s" is 0
    if clean_table:
        merged_df = merged_df[merged_df["A(Bq)@cd = 0s"] != 0]
        merged_df = merged_df[merged_df["t1/2"] < 1e9]
    
    # Save the updated workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
        merged_df.to_excel(writer, sheet_name='Summary_Act', index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook['Summary_Act']
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

In [2]:
#============= convert all half-lives to seconds (Created in Step 4) =============#
def HalfLife_Unit_Factor(unit):
    fac = 0
    if unit == "s":
        fac = 1;
    if unit == "m":
        fac = 60;
    if unit == "h":
        fac = 60*60;
    if unit == "d":
        fac = 24*60*60;
    if unit == "y":
        fac = 365*24*60*60;
    return fac

### Calculate A(Bq) with different cd time

In [3]:
excel_path = "example/DoesCal.xlsx"
df = pd.read_excel(excel_path, sheet_name='Summary_Act')

t = "1days"
df['t1/2_seconds'] = df.apply(lambda row: row['t1/2'] * HalfLife_Unit_Factor(row['unit']), axis=1)
match = re.match(r'(\d+\.?\d*)\s*([a-zA-Z]+)', t);
t_value = float(match.group(1))
t_unit = match.group(2)
t_second = t_value * HalfLife_Unit_Factor(t_unit[0])
df[f'A(Bq)@t={t}'] = df.apply(lambda row: row['A(Bq)@cd = 0s'] * np.exp(-0.693 * t_second / row['t1/2_seconds']) if row['t1/2_seconds'] > 0 else 0, axis=1)


ValueError: Worksheet named 'Summary_Act' not found

In [19]:
def Cal_A_cd(excep_path, t):
    df = pd.read_excel(excel_path, sheet_name='Summary_Act')
    df['t1/2_seconds'] = df.apply(lambda row: row['t1/2'] * HalfLife_Unit_Factor(row['unit']), axis=1)
    match = re.match(r'(\d+\.?\d*)\s*([a-zA-Z]+)', t);
    t_value = float(match.group(1))
    t_unit = match.group(2)
    t_seconds = t_value * HalfLife_Unit_Factor(t_unit[0])
    df[f'A(Bq)@t={t}'] = df.apply(
                                   lambda row: round(row['A(Bq)@cd = 0s'] * np.exp(-0.693 * t_seconds / row['t1/2_seconds']), 2) 
                                   if row['t1/2_seconds'] > 0 else 0, axis=1)
    
    df = df.drop(columns=['t1/2_seconds'])
    # Save the updated workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
        df.to_excel(writer, sheet_name='Summary_Act', index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook['Summary_Act']
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)


In [20]:
excel_path = "example/DoesCal.xlsx"
#summarize(excel_path)
Cal_A_cd(excel_path, '13 hour')